In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split

/home/shunguo/SDE/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## load data

In [2]:
test_df = pd.read_csv('data/test.csv')
train_df = pd.read_csv('data/train.csv')

## make xgboost DMatrix

In [8]:
train, valid = train_test_split(train_df, test_size = 0.33, random_state=1)

y_train = train.label
x_train = train.drop(['label'], axis=1)

y_valid = valid.label
x_valid = valid.drop(['label'], axis=1)

xgb_train = xgb.DMatrix(x_train, label=y_train)
xgb_valid = xgb.DMatrix(x_valid, label=y_valid)
xgb_test = xgb.DMatrix(test_df)

In [9]:
print x_train.shape, x_valid.shape

(28140, 784) (13860, 784)


## xgb params

In [ ]:
params={
    'booster':'gbtree',
    'objective': 'multi:softmax', #多分类的问题
    'num_class':10, # 类别数，与 multisoftmax 并用
    
    'eta': 0.3, # 如同学习率 scikit-learn:eta –> learning_rate 
    'max_depth':6, # 构建树的深度，越大越容易过拟合
    'colsample_bytree':0.3, # 生成树时进行的列采样
    'seed':400,
    
    'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
    'nthread':4,# cpu 线程数
}

watchlist = [(xgb_train, 'train'), (xgb_valid, 'valid')]
clf = xgb.train(params, xgb_train, 1000, watchlist, early_stopping_rounds=100, verbose_eval=10)

[0]	train-merror:0.132942	valid-merror:0.161688
Multiple eval metrics have been passed: 'valid-merror' will be used for early stopping.

Will train until valid-merror hasn't improved in 100 rounds.
[10]	train-merror:0.026297	valid-merror:0.060534
[20]	train-merror:0.007747	valid-merror:0.044517
[30]	train-merror:0.001741	valid-merror:0.037229
[40]	train-merror:0.000107	valid-merror:0.033189
[50]	train-merror:0	valid-merror:0.030808
[60]	train-merror:0	valid-merror:0.030375
[70]	train-merror:0	valid-merror:0.029365
[80]	train-merror:0	valid-merror:0.02785
[90]	train-merror:0	valid-merror:0.027489
[100]	train-merror:0	valid-merror:0.027345
[110]	train-merror:0	valid-merror:0.027128
[120]	train-merror:0	valid-merror:0.026768
[130]	train-merror:0	valid-merror:0.026768
[140]	train-merror:0	valid-merror:0.026623
[150]	train-merror:0	valid-merror:0.026407
[160]	train-merror:0	valid-merror:0.026696
[170]	train-merror:0	valid-merror:0.026335
[180]	train-merror:0	valid-merror:0.026263
[190]	trai

In [11]:
pre_test = clf.predict(xgb_test)
submit_df = pd.read_csv('data/sample_submission.csv')
submit_df.Label = pre_test.astype(int)
submit_df.to_csv('data/xgboost.csv',index=None,encoding='utf-8')
submit_df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
